In [2]:
import os

In [3]:
%pwd

'c:\\Users\\seros\\OneDrive\\Desktop\\end-to-end-ml-project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\seros\\OneDrive\\Desktop\\end-to-end-ml-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float 
    l1_ratio: float
    target_column: str  

In [7]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories
from pathlib import Path
import os

class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH ):

        # Get the project root (current working directory after os.chdir)
        project_root = Path(os.getcwd())
        
        # Construct absolute paths
        config_file_path = project_root / config_file_path
        params_file_path = project_root / params_file_path
        schema_file_path = project_root / schema_file_path

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)        
        

        create_directories([self.config.artifacts_root])



    def get_model_trainer_config(self) -> ModelTrainerConfig:
        model_trainer_config = self.config.model_trainer

        create_directories([model_trainer_config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(model_trainer_config.root_dir),
            train_data_path=Path(model_trainer_config.train_data_path),
            test_data_path=Path(model_trainer_config.test_data_path),
            model_name=model_trainer_config.model_name,
            alpha=self.params.ElasticNet.alpha,
            l1_ratio=self.params.ElasticNet.l1_ratio,
            target_column=self.schema.TARGET_COLUMN.name
        )

        return model_trainer_config

In [8]:
import os
import pandas as pd
from ML_Project.utils import logger
from sklearn.linear_model import ElasticNet
import joblib

[2026-02-09 18:21:07,490: INFO: utils : NumExpr defaulting to 8 threads.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        logger.info("Loading training data...")
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        logger.info("Splitting features and target variable...")
        X_train = train_data.drop(columns=[self.config.target_column])
        X_test = test_data.drop(columns=[self.config.target_column])
        
        
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        logger.info("Training the ElasticNet model...")
        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio)
        model.fit(X_train, y_train)

        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        logger.info(f"Saving the trained model to {model_path}...")
        joblib.dump(model, model_path)    

In [10]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(f"An error occurred during model training: {e}")    

[2026-02-09 18:21:10,945: INFO: common : yaml file: c:\Users\seros\OneDrive\Desktop\end-to-end-ml-project\config\config.yaml loaded successfully]
[2026-02-09 18:21:10,953: INFO: common : yaml file: c:\Users\seros\OneDrive\Desktop\end-to-end-ml-project\params.yaml loaded successfully]
[2026-02-09 18:21:10,971: INFO: common : yaml file: c:\Users\seros\OneDrive\Desktop\end-to-end-ml-project\schema.yaml loaded successfully]
[2026-02-09 18:21:10,973: INFO: common : created directory at: artifacts]
[2026-02-09 18:21:10,975: INFO: common : created directory at: artifacts/model_trainer]
[2026-02-09 18:21:10,976: INFO: 3022631580 : Loading training data...]
[2026-02-09 18:21:11,033: INFO: 3022631580 : Splitting features and target variable...]
[2026-02-09 18:21:11,048: INFO: 3022631580 : Training the ElasticNet model...]
[2026-02-09 18:21:11,056: INFO: 3022631580 : Saving the trained model to artifacts\model_trainer\model.joblib...]
